# Задача
Дан ряд Тейлора для синусной интегральной функции:
$$
\text{Si}(x) = \sum_{n=0}^{\infty} \frac{(-1)^n x^{2n+1}}{(2n+1)(2n+1)!}
$$

### Нахождение отношения последующих членов ряда

Общий член ряда для $\text{Si}(x)$ имеет вид:

$$
a_n = \frac{(-1)^n x^{2n+1}}{(2n+1) \cdot (2n+1)!}
$$

Следующий член:

$$
a_{n+1} = \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!}
$$

Теперь вычислим отношение $q_n(x) = \frac{a_{n+1}}{a_n}$, показывая все преобразования шаг за шагом.

$$
q_n(x) = \frac{a_{n+1}}{a_n} = \frac{ \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!} }{ \frac{(-1)^n x^{2n+1}}{(2n+1) \cdot (2n+1)!} } = \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!} \cdot \frac{(2n+1) \cdot (2n+1)!}{(-1)^n x^{2n+1}}
$$

Сначала упростим знаки и степени $x$:

$$
= (-1)^{n+1} \cdot \frac{1}{(-1)^n} \cdot \frac{x^{2n+3}}{x^{2n+1}} \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!}
$$

$$
= (-1)^{(n+1) - n} \cdot x^{2} \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!} = -x^2 \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!}
$$

Теперь разберёмся с факториалами. Мы знаем, что:

$$
(2n+3)! = (2n+3) \cdot (2n+2) \cdot (2n+1)!
$$

Поэтому:

$$
\frac{(2n+1)!}{(2n+3)!} = \frac{(2n+1)!}{(2n+3) \cdot (2n+2) \cdot (2n+1)!} = \frac{1}{(2n+3) \cdot (2n+2)}
$$

(Здесь $(2n+1)!$ в числителе и знаменателе сокращается полностью, оставляя только обратные множители от $(2n+3)$ и $(2n+2)$.)

Подставляем это обратно:

$$
q_n(x) = -x^2 \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{1}{(2n+3) \cdot (2n+2)} = -x^2 \cdot \frac{(2n+1)}{(2n+3) \cdot (2n+3) \cdot (2n+2)}
$$

$$
= -x^2 \cdot \frac{(2n+1)}{(2n+2) \cdot (2n+3)^2}
$$

$$
= -\frac{x^2 (2n+1)}{(2n+2) (2n+3)^2}
$$

Таким образом, добавочный член $a_{n+1}$ можно получить как $a_{n+1} = q_n(x) \cdot a_n$, где $q_n(x)$ выражено через предыдущий член без полного перерасчёта факториалов. Это удобно для итеративных вычислений ряда, чтобы избежать больших чисел.

In [41]:
def ratio_series(x: float, n: int) -> float:
    """
    q_n(x) - отношения последующего члена к предыдущему
    :param x: аргумент
    :param n: индекс члена ряда
    """
    return -(
            x**2 * (2*n + 1)
    ) / (
            (2*n + 2) * (2*n + 3)**2
    )

In [42]:
def series(x: float, eps: float = 10 ** -6) -> float:
    """
    S_n(x) - реализация с использованием заранее вычисленного отношения последующего члена
    :param x: аргумент
    :param eps: точность
    """
    n = 0
    series_member_old = x # Рассчитанное значение для n = 0
    sum_series = x
    while True:
        n += 1
        series_member_new = series_member_old * ratio_series(x, n - 1)
        sum_series += series_member_new
        if abs(series_member_new - series_member_old) < eps: break
        series_member_old = series_member_new
    return sum_series

Вычисление суммы в предложенных преподавателем точках x = 0.8; x = 2; x = 3.2

In [43]:
for x in [0.8, 2, 3.2]: print(f'{series(x):.6f}')

0.772096
1.605413
1.851401


Нативная реализация подсчета суммы (не оптимизированная)

In [44]:
import math

def native_series(x: float, eps: float = 10 ** -6) -> float:
    """
    S_n(x) - нативная реализация суммы
    :param x: аргумент
    :param eps: точность
    """
    n = 0
    series_member_old = x
    sum_series = x
    while True:
        n += 1
        series_member_new = ((-1)**n * x**(2*n + 1)) / ((2*n + 1) * math.factorial(2*n + 1))
        sum_series += series_member_new
        if abs(series_member_new - series_member_old) < eps: break
        series_member_old = series_member_new
    return sum_series

Сравнение моей реализации с нативной в предложенных точках

In [45]:
print('Моя\t\t\tНативная')
for x in [0.8, 2, 3.2]: print(f'{series(x):.6f}\t{native_series(x):.6f}')

Моя			Нативная
0.772096	0.772096
1.605413	1.605413
1.851401	1.851401


### Построение таблицы значений

In [46]:
import pandas
from dataclasses import dataclass

@dataclass(frozen=True)
class Point:
    x: float
    y: float
    
def get_tab_x(a: float = 0, b: float = 4, n: int = 5)-> list[float]:
    """
    :param a: начальная точка отрезка a < b
    :param b: конечная точка отрезка b > a
    :param n: количество интервалов 
    :return: список точек
    """
    h = (b - a) / n
    tab_x = [a]
    while len(tab_x) < n: tab_x.append(tab_x[-1] + h)
    tab_x.append(b)
    return tab_x

tabulate_series = [Point(x, series(x)) for x in get_tab_x()]
x_values = [point.x for point in tabulate_series]
y_values = [point.y for point in tabulate_series]

pandas.DataFrame([x_values, y_values], index=['x', 'f(x)'])

,0,1,2,3,4,5
x,0.0,0.800000,1.60000,2.400000,3.200000,4.000000
f(x),0.0,0.772096,1.38918,1.752486,1.851401,1.758203


### Функция для вычисления значения интерполяционного полинома Лагранжа

In [47]:
def lagrange_polynomial(x: float, points: list[Point]) -> float:
    n = len(points) - 1
    result_sum = 0
    for i in range(n):
        result_mult = 1
        for j in range(n):
            if j == i: continue
            result_mult *= (x - points[j].x) / (points[i].x - points[j].x)
        result_sum += points[i].y * result_mult
    return result_sum
    

In [56]:
x_array = get_tab_x(n=5)

result_table = {
    'x_i': x_array,
    'S(x_i)': [series(x) for x in x_array],
    'L(x_i)': [lagrange_polynomial(x, tabulate_series) for x in x_array],
}

pandas.DataFrame(result_table)

x_i,S(x_i),L(x_i)
0.0,0.000000,0.000000
0.8,0.772096,0.772096
1.6,1.389180,1.389180
2.4,1.752486,1.752486
3.2,1.851401,1.851401
4.0,1.758203,1.763475


In [73]:
x_array = get_tab_x(n=10)
series_array = [series(x) for x in x_array]
series_tabulate = [Point(x, s) for x, s in zip(x_array, series_array)]
lp_array = [lagrange_polynomial(x, tabulate_series) for x in x_array]

result_table = {
    '~x_i': x_array,
    'S(~x_i)': series_array,
    'L(~x_i)': lp_array,
    'Погрешность': [abs(s - l) for s, l in zip(series_array, lp_array)]
}

pandas.DataFrame(result_table)

~x_i,S(~x_i),L(~x_i),Погрешность
0.0,0.000000,0.000000,0.000000e+00
0.4,0.396461,0.395808,6.539219e-04
0.8,0.772096,0.772096,0.000000e+00
1.2,1.108047,1.108254,2.065786e-04
1.6,1.389180,1.389180,0.000000e+00
2.0,1.605413,1.605285,1.281831e-04
2.4,1.752486,1.752486,4.440892e-16
2.8,1.832097,1.832211,1.147349e-04
3.2,1.851401,1.851401,2.220446e-16
3.6,1.821948,1.822503,5.546545e-04


### Погрешности при увеличении числа интервалов

In [77]:
n_array = [i for i in range(1, 50)]
m_array = [i*2 for i in n_array]

middle_error_rate = []
for n, m in zip(n_array, m_array):
    x_array = get_tab_x(n=n)
    series_array = [series(x) for x in x_array]
    series_tabulate = [Point(x, s) for x, s in zip(x_array, series_array)]
    
    x_m_array = get_tab_x(n=m)[:-1]
    series_array = [series(x) for x in x_m_array]
    lp_array = [lagrange_polynomial(x, series_tabulate) for x in x_m_array]
    error_rate = [abs(s - l) for s, l in zip(series_array, lp_array)]
    
    middle_error_rate.append(sum(error_rate) / len(error_rate))

result_table = {
    'n': n_array,
    'Средняя погрешность': middle_error_rate,
}

pandas.DataFrame(result_table)

n,Средняя погрешность
1,8.027065e-01
2,1.757109e-01
3,1.509458e-02
4,1.007685e-02
5,1.658073e-04
6,3.120210e-04
7,6.868896e-06
8,6.447745e-06
9,1.637124e-07
10,9.565627e-08
